In [2]:
import cv2
import os
from concurrent.futures import ProcessPoolExecutor

# Paths to your images and annotations
image_dir = '/mnt/storage/kilsar_jainil/detected_unzipped/detected/images'  # Directory containing original images
annotation_dir = '/mnt/storage/kilsar_jainil/detected_unzipped/detected/labels'  # Directory containing YOLO annotation files
output_dir = '/home/spn7g/Weapon detection scripts/data/labled_box_data'  # Directory to save images with bounding boxes

# Number of CPU cores to use
num_cores = 50

# Make sure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load class names (modify according to your classes)
class_names = ['class1', 'class2', 'class3']  # Adjust this list to your actual class names

# Helper function to denormalize YOLO coordinates
def denormalize_bbox(x_center, y_center, width, height, img_width, img_height):
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height
    x1 = int(x_center - (width / 2))
    y1 = int(y_center - (height / 2))
    x2 = int(x_center + (width / 2))
    y2 = int(y_center + (height / 2))
    return x1, y1, x2, y2

# Function to process a single image and its annotations
def process_image(image_name):
    try:
        # Load image
        image_path = os.path.join(image_dir, image_name)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image {image_name}")
            return
        
        img_height, img_width, _ = img.shape

        # Load corresponding annotation
        annotation_path = os.path.join(annotation_dir, os.path.splitext(image_name)[0] + '.txt')
        if not os.path.exists(annotation_path):
            print(f"Annotation for {image_name} not found, skipping.")
            return

        with open(annotation_path, 'r') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Invalid annotation line in {annotation_path}: {line.strip()}")
                    continue
                
                class_id, x_center, y_center, width, height = map(float, parts)
                x1, y1, x2, y2 = denormalize_bbox(x_center, y_center, width, height, img_width, img_height)
                
                # Draw the bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box, 2px thick
                class_name = class_names[int(class_id)]
                cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Save the image with bounding boxes using the same name as the input image
        output_path = os.path.join(output_dir, image_name)
        cv2.imwrite(output_path, img)
        print(f"Processed and saved {image_name}")

    except Exception as e:
        print(f"Error processing {image_name}: {e}")

# List of images to process
image_list = [img_name for img_name in os.listdir(image_dir) if img_name.endswith(('.jpg', '.png'))]

# Process images in parallel using ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=num_cores) as executor:
    executor.map(process_image, image_list)

print(f"Bounding box images saved in {output_dir}")


KeyboardInterrupt: 

Annotation for Fringe_S01E19_P229_1.jpg.jpg not found, skipping.


In [1]:
import os

# Get the number of CPU cores
cpu_cores = os.cpu_count()

print(f"Number of CPU cores: {cpu_cores}")


Number of CPU cores: 96
